In [21]:
import pandas as pd
import requests
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows

In [22]:
# Specify the parameters required to import annual GDP, inflation and interest rate data for the USA going back to 2000 from the World Bank API
url = "http://api.worldbank.org/v2/country/USA/indicator/"
indicators = ["NY.GDP.MKTP.CD", "FP.CPI.TOTL.ZG", "FR.INR.RINR"]
start_year = "2000"
end_year = "2022"
params = {
    "format": "json",
    "date": f"{start_year}:{end_year}"
}

In [23]:
# Fetch data from the API
data = {}
for indicator in indicators:
    response = requests.get(url + indicator, params=params)
    json_data = response.json()[1]
    for item in json_data:
        year = item["date"]
        value = item["value"]
        if year not in data:
            data[year] = {}
        data[year][indicator] = value


In [24]:
# Convert data to a pandas dataframe called df
df = pd.DataFrame.from_dict(data, orient="index")
df.index.name = "Year"
df.columns = ["GDP", "Inflation", "Interest Rate"]

In [25]:
# Create a new workbook which will be used as a staging post to collate all data to be used in my analysis
workbook = openpyxl.Workbook()

In [26]:
# Rename the default sheet to "Economic_data"
worksheet1 = workbook.active
worksheet1.title = "Economic_data"

In [27]:
# Create a new sheet and name it "Stockmarket_data"
worksheet2 = workbook.create_sheet(title="Stockmarket_data")
# Create a new sheet and name it "Company_data"
worksheet3 = workbook.create_sheet(title="Company_data")

In [28]:
# Rename the workbook to "Data_Source"
workbook.title = "Data_Source"

In [29]:
# Export df to the "Economic_data" sheet
for r in dataframe_to_rows(df, header=True):
    worksheet1.append(r)

In [30]:
# Save the workbook to a file
workbook.save("data_source.xlsx")